<a href="https://colab.research.google.com/github/MasaoAsano/kagglexMentorshipProgram/blob/main/visionApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-vision

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir "/content/gdrive/My Drive/cloudVisionTrial"
!mkdir "/content/gdrive/My Drive/cloudVisionTrial/images"

In [ ]:
%cd "./gdrive/My Drive/cloudVisionTrial"

In [ ]:
import glob

src_files = glob.glob('/content/gdrive/My Drive/cloudVisionTrial/images/*.jpeg')
src_files.sort()

for f in src_files:
    print(f)

In [ ]:
import os
import os.path
import errno

from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

cred_path = '/content/gdrive/My Drive/cloudVisionTrial/visionapitrial-401002-dff8a7a32b09.json'

if os.path.exists(cred_path) == False:
  raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), cred_path)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = cred_path


In [ ]:
import io
from google.cloud import vision
from google.cloud.vision_v1 import types

objects = {}

client = vision.ImageAnnotatorClient()

for file in src_files:
  with io.open(file, 'rb') as image_file:
      content = image_file.read()

  image = types.Image(content=content)
  res = client.object_localization(
        image=image).localized_object_annotations
  objects[file] = res

In [ ]:
from matplotlib import pyplot as plt

from PIL import Image
from PIL import ImageDraw

def draw_object_boxes(f, objects):
    im = Image.open(f)
    w, h = im.size
    draw = ImageDraw.Draw(im)
    for obj in objects:
      if(obj.name=='License plate'):
        color = '#ffffff'
        box = [(v.x * w, v.y * h) for v in obj.bounding_poly.normalized_vertices]

        left = min([v.x * w for v in obj.bounding_poly.normalized_vertices])
        right = max([v.x * w for v in obj.bounding_poly.normalized_vertices])
        lower = min([v.y * h for v in obj.bounding_poly.normalized_vertices])
        upper = max([v.y * h for v in obj.bounding_poly.normalized_vertices])

        ImageDraw.Draw(im).rectangle(((left, upper), (right, lower)), fill=color)
    return im

# for file in files:
for file in src_files:
  plt.figure(figsize=(10,6))
  plt.imshow(draw_object_boxes(file, objects[file]))
plt.show()
